## Notebook 5: Tracking Wildfires from Space

**The 2019-2020 Australian fires burned an area the size of South Korea - we can watch it happen from space.**

The "Black Summer" bushfire season was one of the worst natural disasters in Australian history:
- **46 million acres** burned (186,000 km²)
- **Over 1 billion animals** estimated killed
- **34 people** died directly from the fires
- Smoke circled the globe and was visible from space

In this notebook, we'll:

- Use MODIS satellite data to detect active fires
- Map the progression of fires from November 2019 through February 2020
- Create an animation showing the fire season unfold

This is the same technology used by fire agencies worldwide for near-real-time fire monitoring.

## Setup

Same initialization as previous notebooks.

In [1]:
%pip install -q geemap folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee
from google.cloud import storage
from IPython.display import Image, display

# Initialize Earth Engine
PROJECT = "eeps-geospatial"
BUCKET = "wustl-eeps-edc"
ee.Initialize(project=PROJECT)

import geemap.foliumap as geemap

print("Ready!")

Ready!


## How satellites detect fires

Satellites detect active fires by looking for **thermal anomalies** - spots that are significantly hotter than their surroundings.

**MODIS (Moderate Resolution Imaging Spectroradiometer):**
- Flies on NASA's Terra and Aqua satellites
- Scans the entire Earth every 1-2 days
- Has special thermal infrared bands sensitive to fire temperatures (3.9 μm and 11 μm)
- Can detect fires as small as 1000 m² (about 1/4 acre)

**The MOD14A1 product provides:**
- `FireMask`: Classification of each pixel (fire, no fire, cloud, water, etc.)
- `MaxFRP`: Maximum Fire Radiative Power - a measure of fire intensity in megawatts

**Fire Radiative Power (FRP)** correlates with:
- Rate of fuel consumption
- Smoke emissions
- Overall fire intensity

## Define area of interest

We'll focus on Southeast Australia - New South Wales and Victoria were the hardest hit during the Black Summer.

In [3]:
# Southeast Australia - NSW and Victoria
australia_se = ee.Geometry.Rectangle([145, -38, 154, -28])

# Key locations for reference
landmarks = {
    "Sydney": [151.2093, -33.8688],
    "Melbourne": [144.9631, -37.8136],
    "Canberra": [149.1300, -35.2809],
}

# Center point for maps
center_lat = -33.5
center_lon = 149.5

print("Area of interest: Southeast Australia (NSW & Victoria)")

Area of interest: Southeast Australia (NSW & Victoria)


## Load MODIS Active Fire data

We'll use the MOD14A1 product which provides daily fire detections at 1km resolution.

**Timeline of Black Summer:**
- September 2019: Early fires begin
- November 2019: Fires intensify significantly
- December 2019 - January 2020: Peak intensity, catastrophic conditions
- February 2020: Heavy rains finally bring relief

In [4]:
# MODIS Thermal Anomalies/Fire product
mod14 = ee.ImageCollection("MODIS/061/MOD14A1")

# Fire season dates
fire_start = "2019-11-01"
fire_end = "2020-02-29"

# Filter to our region and time period
fires = (
    mod14
    .filterDate(fire_start, fire_end)
    .filterBounds(australia_se)
    .select(['FireMask', 'MaxFRP'])
)

print(f"Loaded {fires.size().getInfo()} days of fire data")

Loaded 120 days of fire data


## Process fire detections

The FireMask band uses these values:
- 0-2: Not processed or missing data
- 3: Water
- 4: Cloud
- 5: No fire (land)
- 6: Unknown
- 7: Low confidence fire
- 8: Nominal confidence fire
- 9: High confidence fire

We'll focus on values 7-9 (fire detections) and use MaxFRP for intensity.

In [5]:
def extract_fires(img):
    """Extract fire pixels and their intensity."""
    fire_mask = img.select('FireMask')
    frp = img.select('MaxFRP')
    
    # Fire pixels are values 7, 8, 9 (low, nominal, high confidence)
    is_fire = fire_mask.gte(7).And(fire_mask.lte(9))
    
    # Mask FRP to only fire pixels, scale by 0.1 to get MW
    fire_frp = frp.multiply(0.1).updateMask(is_fire)
    
    return fire_frp.set('system:time_start', img.get('system:time_start'))

# Apply to all images
fire_frp = fires.map(extract_fires)

# Create composite of maximum fire intensity over entire period
fire_max = fire_frp.max().clip(australia_se)

print("Fire detections processed")

Fire detections processed


## Visualize cumulative fire extent

First, let's see everywhere that burned during the entire Black Summer season.

In [6]:
# Create map
m = geemap.Map(center=[center_lat, center_lon], zoom=6)

# Fire intensity palette: yellow (low) to red (high)
fire_palette = [
    '#FFFF00',  # Yellow - low intensity
    '#FFCC00',  # Gold
    '#FF9900',  # Orange
    '#FF6600',  # Dark orange
    '#FF3300',  # Red-orange
    '#FF0000',  # Red
    '#CC0000',  # Dark red - high intensity
]

fire_vis = {
    'min': 0,
    'max': 500,  # FRP in MW
    'palette': fire_palette
}

# Add fire layer
m.addLayer(fire_max, fire_vis, 'Max Fire Intensity (Nov 2019 - Feb 2020)')

# Add landmarks
for name, coords in landmarks.items():
    point = ee.Geometry.Point(coords)
    m.addLayer(point, {'color': 'blue'}, name)

# Add region boundary
m.addLayer(australia_se, {'color': 'white'}, 'Study Area', False)

m.add_colorbar(fire_vis, label='Fire Radiative Power (MW)', layer_name='Max Fire Intensity (Nov 2019 - Feb 2020)')

m

## Monthly fire progression

Let's create monthly composites to see how the fires spread over time.

In [7]:
# Define months of the fire season
months = [
    ('2019-11-01', '2019-11-30', 'November 2019'),
    ('2019-12-01', '2019-12-31', 'December 2019'),
    ('2020-01-01', '2020-01-31', 'January 2020'),
    ('2020-02-01', '2020-02-29', 'February 2020'),
]

def get_monthly_fires(start, end, label):
    """Get maximum fire intensity for a month."""
    monthly = (
        fire_frp
        .filterDate(start, end)
        .max()
        .clip(australia_se)
    )
    return monthly.set('month', label)

# Create monthly composites
monthly_fires = [get_monthly_fires(start, end, label) for start, end, label in months]

print(f"Created {len(monthly_fires)} monthly fire composites")

Created 4 monthly fire composites


In [8]:
# Compare December (peak) vs November (early)
nov_fires = monthly_fires[0]
dec_fires = monthly_fires[1]

left_layer = geemap.ee_tile_layer(nov_fires, fire_vis, 'November 2019')
right_layer = geemap.ee_tile_layer(dec_fires, fire_vis, 'December 2019')

m2 = geemap.Map(center=[center_lat, center_lon], zoom=6)
m2.split_map(left_layer, right_layer)
m2

## Animate the fire season

Let's create an animation showing how fires spread week by week.

In [9]:
# Create weekly composites for animation with satellite background
import datetime

# Get a cloud-free Landsat composite as background
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
background = (
    landsat
    .filterBounds(australia_se)
    .filterDate('2019-06-01', '2019-10-31')  # Before the fires
    .filter(ee.Filter.lt('CLOUD_COVER', 20))
    .median()
    .clip(australia_se)
)

# True color bands, stretched for visibility
bg_vis = background.select(['B4', 'B3', 'B2']).multiply(255 * 2.5).clamp(0, 255).toUint8()

# Generate weekly date ranges
start_date = datetime.date(2019, 11, 1)
end_date = datetime.date(2020, 2, 29)
weekly_images = []

current = start_date
while current < end_date:
    week_end = min(current + datetime.timedelta(days=6), end_date)
    
    weekly_fire = (
        fire_frp
        .filterDate(current.isoformat(), week_end.isoformat())
        .max()
        .clip(australia_se)
    )
    
    # Create fire overlay: bright orange/red for fires
    fire_mask = weekly_fire.gt(0)
    fire_intensity = weekly_fire.clamp(0, 300).divide(300)
    
    # Fire colors: orange to red based on intensity
    fire_r = ee.Image(255).clip(australia_se)
    fire_g = ee.Image(150).subtract(fire_intensity.multiply(100)).clip(australia_se)
    fire_b = ee.Image(0).clip(australia_se)
    fire_rgb = fire_r.addBands(fire_g).addBands(fire_b).toUint8().updateMask(fire_mask)
    
    # Blend: use background where no fire, fire colors where fire
    blended = bg_vis.blend(fire_rgb)
    weekly_images.append(blended)
    
    current = week_end + datetime.timedelta(days=1)

print(f"Created {len(weekly_images)} weekly composites for animation")

Created 18 weekly composites for animation


In [10]:
# Convert to ImageCollection
weekly_collection = ee.ImageCollection.fromImages(weekly_images)

# Animation parameters - simpler for RGB
animation_params = {
    'region': australia_se,
    'dimensions': 500,
    'crs': 'EPSG:4326',
    'framesPerSecond': 2,
}

# Generate animation URL
gif_url = weekly_collection.getVideoThumbURL(animation_params)
print("Animation URL generated!")

Animation URL generated!


In [11]:
print("Black Summer Fire Progression: November 2019 - February 2020")
print("Watch the fires intensify through December-January, then fade in February with rain.")
display(Image(url=gif_url))

Black Summer Fire Progression: November 2019 - February 2020
Watch the fires intensify through December-January, then fade in February with rain.


## Burned area analysis

MODIS also provides a monthly burned area product (MCD64A1) that maps the extent of burning more precisely than active fire detections.

In [12]:
# Load MODIS Burned Area product
mcd64 = ee.ImageCollection("MODIS/061/MCD64A1")

# Get burned area for the fire season
burned = (
    mcd64
    .filterDate('2019-11-01', '2020-02-29')
    .filterBounds(australia_se)
    .select('BurnDate')
)

# Any pixel with a valid burn date burned during this period
burned_mask = burned.max().gt(0).clip(australia_se)

print(f"Loaded burned area data")

Loaded burned area data


In [13]:
# Create map showing burned areas
m3 = geemap.Map(center=[center_lat, center_lon], zoom=6)

# Add burned area in red
m3.addLayer(
    burned_mask.selfMask(), 
    {'palette': ['FF4500']},  # Orange-red
    'Burned Area (Nov 2019 - Feb 2020)'
)

# Add fire intensity on top
m3.addLayer(fire_max, fire_vis, 'Fire Intensity', False)

# Add landmarks
for name, coords in landmarks.items():
    point = ee.Geometry.Point(coords)
    m3.addLayer(point, {'color': 'cyan'}, name)

m3

## Calculate burned area statistics

In [14]:
# Calculate total burned area
burned_area = burned_mask.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=australia_se,
    scale=500,  # MODIS resolution
    maxPixels=1e9
)

# Convert to sq km
area_sqkm = ee.Number(burned_area.get('BurnDate')).divide(1e6)

print(f"Burned area in study region: {area_sqkm.getInfo():,.0f} sq km")
print(f"\nFor context:")
print(f"  - South Korea: ~100,000 sq km")
print(f"  - Belgium: ~30,500 sq km")
print(f"  - Massachusetts: ~27,300 sq km")

Burned area in study region: 52,290 sq km

For context:
  - South Korea: ~100,000 sq km
  - Belgium: ~30,500 sq km
  - Massachusetts: ~27,300 sq km


## What patterns do you notice?

Looking at the maps and animation:

- **Coastal concentration**: Many fires occurred in the Great Dividing Range and coastal forests
- **December-January peak**: The most intense burning occurred during these months
- **Proximity to cities**: Fires came dangerously close to Sydney, Canberra, and Melbourne
- **Rapid spread**: In the animation, you can see how quickly fires expanded during bad fire weather

The Black Summer was exacerbated by extreme drought, record temperatures, and strong winds - a combination climate scientists warn will become more common.

## Try it yourself

Ideas to explore:

1. **California 2020**: Look at August-September 2020 when record fires burned in California. Change the AOI to `ee.Geometry.Rectangle([-125, 32, -114, 42])`

2. **Amazon fires**: Examine the 2019 Amazon fire season (August-September 2019). AOI: `ee.Geometry.Rectangle([-70, -15, -45, 0])`

3. **Compare fire seasons**: How does Black Summer compare to a "normal" Australian fire season? Try the same analysis for 2018-2019.

4. **Fire weather correlation**: Download temperature and precipitation data - do the biggest fire days correspond to hot, dry, windy conditions?

5. **Recovery monitoring**: Use NDVI to see how vegetation is recovering in burned areas. Compare 2020 NDVI to 2019.

6. **Smoke plumes**: MODIS also detects aerosols. Look at the `MODIS/061/MOD04_L2` product to see smoke spreading across the Pacific.